In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"  # or choose another backend if you wish
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, DatetimeTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.io import output_notebook
import sklearn
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import datetime
import keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Flatten, Dense, Dropout, Input

Using plaidml.keras.backend backend.


In [31]:
output_notebook()
start_date = datetime.datetime(2005, 2, 1)
end_date = datetime.datetime(2020, 3, 30)

Loading BokehJS ...

In [3]:
def date_filter(df):
    return df.loc[(df.index >= start_date) & (df.index <= end_date)]

In [4]:
def load_weather_data():
    df = pd.read_csv('../data/central_park_weather.csv', dtype='object')
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['weekday'] = df.DATE.dt.weekday
    df['week'] = df.DATE.dt.week
    df['month'] = df.DATE.dt.month
    df['year'] = df.DATE.dt.year
    df = df[df.DATE >= '20010101']
    df['TMAX'] = df['TMAX'].astype('float')
    df['TMIN'] = df['TMIN'].astype('float')
    df['PRCP'] = df['PRCP'].astype('float')
    return df[[
        'weekday', 
        'week', 
        'month', 
        'PRCP',
        'TMAX',
        'TMIN',
        'DATE',
    ]].set_index('DATE').sort_index()

In [5]:
weather_data = load_weather_data()

In [6]:
def load_pal_data():
    df = pd.read_csv('../data/nyiso_pal_master.csv')
    df['Time Stamp'] = pd.to_datetime(df['Time Stamp'])
    return df[[
        'Time Stamp',
        'pal_min',
        'pal_max',
        'pal_mean',
    ]].set_index('Time Stamp').sort_index()

In [7]:
weather_data = date_filter(weather_data)
actual_load = load_pal_data()
actual_load = date_filter(actual_load)
print(weather_data.head())
print(weather_data.tail())
print(actual_load.head())
print(actual_load.tail())
print(weather_data.shape)
print(actual_load.shape)

            weekday  week  month  PRCP  TMAX  TMIN
DATE                                              
2005-02-01        1     5      2  0.00  40.0  25.0
2005-02-02        2     5      2  0.00  40.0  28.0
2005-02-03        3     5      2  0.02  41.0  29.0
2005-02-04        4     5      2  0.27  46.0  34.0
2005-02-05        5     5      2  0.00  53.0  38.0
            weekday  week  month  PRCP  TMAX  TMIN
DATE                                              
2020-03-26        3    13      3  0.00  60.0  38.0
2020-03-27        4    13      3  0.00  69.0  50.0
2020-03-28        5    13      3  0.45  54.0  44.0
2020-03-29        6    13      3  0.05  47.0  44.0
2020-03-30        0    14      3  0.04  52.0  41.0
            pal_min  pal_max     pal_mean
Time Stamp                               
2005-02-01      0.0   7154.7  5580.662630
2005-02-02      0.0   7153.9  5535.835517
2005-02-03      0.0   7152.1  5514.954639
2005-02-04      0.0   6890.1  5717.773469
2005-02-05   4299.9   5990.4  5130

In [8]:
merged = actual_load.join(weather_data, how='inner')
print(merged.head())
print(merged.shape)

            pal_min  pal_max     pal_mean  weekday  week  month  PRCP  TMAX  \
2005-02-01      0.0   7154.7  5580.662630        1     5      2  0.00  40.0   
2005-02-02      0.0   7153.9  5535.835517        2     5      2  0.00  40.0   
2005-02-03      0.0   7152.1  5514.954639        3     5      2  0.02  41.0   
2005-02-04      0.0   6890.1  5717.773469        4     5      2  0.27  46.0   
2005-02-05   4299.9   5990.4  5130.559122        5     5      2  0.00  53.0   

            TMIN  
2005-02-01  25.0  
2005-02-02  28.0  
2005-02-03  29.0  
2005-02-04  34.0  
2005-02-05  38.0  
(5513, 9)


In [9]:
merged.isnull().values.any()

False

In [10]:
merged = merged.drop(columns=['pal_min', 'pal_max'])

In [11]:
merged = merged.sample(frac=1)
merged.head()

,pal_mean,weekday,week,month,PRCP,TMAX,TMIN
2008-03-19,5998.350523,2,12,3,0.95,53.0,41.0
2015-03-12,5669.358305,3,11,3,0.00,47.0,36.0
2018-10-20,4809.904138,5,42,10,0.06,62.0,51.0
2009-02-21,5558.342561,5,8,2,0.00,42.0,26.0
2008-08-12,6763.660068,1,33,8,0.00,82.0,60.0


Now, we will normalize the data.

In [12]:
# should only be computed using training data
mean = merged.mean(axis=0)
merged -= mean
std = merged.std(axis=0)
merged /= std
merged.head()

,pal_mean,weekday,week,month,PRCP,TMAX,TMIN
2008-03-19,-0.107443,-0.499887,-0.955294,-1.009380,1.988700,-0.552386,-0.470274
2015-03-12,-0.447271,0.000181,-1.021544,-1.009380,-0.355005,-0.882435,-0.771134
2018-10-20,-1.335034,1.000318,1.032228,1.014257,-0.206981,-0.057313,0.131444
2009-02-21,-0.561944,1.000318,-1.220296,-1.298471,-0.355005,-1.157476,-1.372852
2008-08-12,0.683074,-0.999955,0.435971,0.436075,-0.355005,1.042850,0.672990


In [13]:
train_df = merged.sample(frac=0.8)
labels = train_df['pal_mean'].tolist()
train_df.drop(columns=['pal_mean'], inplace=True)
train_df.head()

,weekday,week,month,PRCP,TMAX,TMIN
2012-07-18,-0.499887,0.170968,0.146984,3.987016,2.032996,1.455224
2017-11-24,0.500249,1.363481,1.303348,-0.355005,-0.717411,-0.831305
2007-09-06,0.000181,0.634723,0.725166,-0.355005,0.932834,1.094193
2011-02-13,1.500386,-1.352798,-1.298471,-0.355005,-0.937443,-1.252508
2006-07-21,0.500249,0.170968,0.146984,4.455757,1.152866,1.274708


In [14]:
main_input = Input(shape=(train_df.shape[1],), name='main_input')

x = Dense(64, activation='relu')(main_input)
x = Dropout(.25)(x)
x = Dense(64, activation='relu')(x)

output = Dense(1, name='output')(x)

model = Model(inputs=[main_input], outputs=output)
model.summary()

INFO:plaidml:Opening device "metal_amd_radeon_rx_580.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 6)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                448       
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
output (Dense)               (None, 1)                 65        
Total params: 4,673
Trainable params: 4,673
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae'])
history = model.fit([train_df], labels,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                   )

Train on 3528 samples, validate on 882 samples
Epoch 1/50
3528/3528 [==============================] - 1s 360us/step - loss: 0.3819 - mean_absolute_error: 0.4569 - val_loss: 0.1471 - val_mean_absolute_error: 0.2926
Epoch 2/50
3528/3528 [==============================] - 0s 113us/step - loss: 0.1817 - mean_absolute_error: 0.3138 - val_loss: 0.1038 - val_mean_absolute_error: 0.2417
Epoch 3/50
3528/3528 [==============================] - 0s 111us/step - loss: 0.1532 - mean_absolute_error: 0.2809 - val_loss: 0.0911 - val_mean_absolute_error: 0.2251
Epoch 4/50
3528/3528 [==============================] - 0s 111us/step - loss: 0.1420 - mean_absolute_error: 0.2746 - val_loss: 0.0880 - val_mean_absolute_error: 0.2219
Epoch 5/50
3528/3528 [==============================] - 0s 112us/step - loss: 0.1307 - mean_absolute_error: 0.2633 - val_loss: 0.0848 - val_mean_absolute_error: 0.2173
Epoch 6/50
3528/3528 [==============================] - 0s 126us/step - loss: 0.1208 - mean_absolute_error: 0.251

In [83]:
source = pd.DataFrame(history.history)
source['epoch'] = source.index + 1
acc_plot = figure(plot_width=800, plot_height=400, x_axis_label='Epoch', y_axis_label='MAE')
acc_plot.circle(x='epoch', y='mean_absolute_error', source=source, size=10, fill_alpha=.5, legend_label='MAE')
acc_plot.line(x='epoch', y='val_mean_absolute_error', source=source, line_width=2, legend_label='Val MAE', alpha=0.5)
show(acc_plot)

In [56]:
loss_plot = figure(plot_width=800, plot_height=400, x_axis_label='Epoch', y_axis_label='Loss')
loss_plot.circle(x='epoch', y='loss', source=source, size=10, fill_alpha=.5, legend_label='Loss')
loss_plot.line(x='epoch', y='val_loss', source=source, line_width=2, legend_label='Val Loss', alpha=0.5)
show(loss_plot)

In [118]:
predictions = model.predict(merged.copy().drop(columns=['pal_mean']))
results = merged.copy()
results['prediction'] = predictions
results['pal_mean'] *= std.pal_mean
results['pal_mean'] += mean.pal_mean
results['prediction'] *= std.pal_mean
results['prediction'] += mean.pal_mean
results['date'] = results.index
results

,pal_mean,weekday,week,month,PRCP,TMAX,TMIN,prediction,date
2008-03-19,5998.350523,-0.499887,-0.955294,-1.009380,1.988700,-0.552386,-0.470274,5689.666016,2008-03-19
2015-03-12,5669.358305,0.000181,-1.021544,-1.009380,-0.355005,-0.882435,-0.771134,5739.512207,2015-03-12
2018-10-20,4809.904138,1.000318,1.032228,1.014257,-0.206981,-0.057313,0.131444,5169.170898,2018-10-20
2009-02-21,5558.342561,1.000318,-1.220296,-1.298471,-0.355005,-1.157476,-1.372852,5549.208496,2009-02-21
2008-08-12,6763.660068,-0.999955,0.435971,0.436075,-0.355005,1.042850,0.672990,6518.923828,2008-08-12
...,...,...,...,...,...,...,...,...,...
2017-09-26,7443.566667,-0.999955,0.833475,0.725166,-0.355005,1.152866,1.154365,6857.965820,2017-09-26
2014-03-08,5270.732986,1.000318,-1.087795,-1.009380,-0.355005,-0.332354,-0.831305,5277.749512,2014-03-08
2014-02-26,6166.520656,-0.499887,-1.154046,-1.298471,-0.280993,-1.762565,-1.733883,6132.306152,2014-02-26
2008-10-06,5696.539373,-1.500023,0.965977,1.014257,-0.355005,0.052703,-0.109244,5482.741699,2008-10-06


In [120]:
results = results.sort_index()
# results = results[-100:]
results

,pal_mean,weekday,week,month,PRCP,TMAX,TMIN,prediction,date
2005-02-01,5580.662630,-0.999955,-1.419048,-1.298471,-0.355005,-1.267492,-1.433024,5868.906250,2005-02-01
2005-02-02,5535.835517,-0.499887,-1.419048,-1.298471,-0.355005,-1.267492,-1.252508,5909.384277,2005-02-02
2005-02-03,5514.954639,0.000181,-1.419048,-1.298471,-0.305664,-1.212484,-1.192336,5959.702637,2005-02-03
2005-02-04,5717.773469,0.500249,-1.419048,-1.298471,0.311101,-0.937443,-0.891477,5849.549805,2005-02-04
2005-02-05,5130.559122,1.000318,-1.419048,-1.298471,-0.355005,-0.552386,-0.650790,5342.416504,2005-02-05
...,...,...,...,...,...,...,...,...,...
2020-03-26,4709.563448,0.000181,-0.889043,-1.009380,-0.355005,-0.167329,-0.650790,5660.777344,2020-03-26
2020-03-27,4544.689236,0.500249,-0.889043,-1.009380,-0.355005,0.327744,0.071272,5474.657715,2020-03-27
2020-03-28,4425.012329,1.000318,-0.889043,-1.009380,0.755171,-0.497378,-0.289759,5206.997559,2020-03-28
2020-03-29,4447.838255,1.500386,-0.889043,-1.009380,-0.231652,-0.882435,-0.289759,5063.610840,2020-03-29


In [122]:
# results = ColumnDataSource(results)
pred_plot = figure(plot_width=1200, plot_height=600, x_axis_label='Date', y_axis_label='Usage')
pred_plot.circle(x='date', y='pal_mean', source=results, size=10, fill_alpha=.5, legend_label='Actual')
pred_plot.triangle(x='date', y='prediction', source=results, size=10, fill_alpha=.5, legend_label='Prediction', color='green')
pred_plot.line(x='date', y='prediction', source=results, alpha=.5, legend_label='Prediction', color='green')
pred_plot.xaxis.formatter=DatetimeTickFormatter()
show(pred_plot)